In [712]:
import math
import random

def CreateInitialPopulation(size, graph, start_attraction, end_attraction, final_path_size):
    """Generates the initial population for the genetic algorithm
    
    Uses 50% chance of random initialization and 50% chance of nearest neighbor initialization

    Args:
        size (int): The size of the list (initial_population) to be returned.
        graph (dict): A dictionary representing the graph of attractions.
        start_attraction (str): The starting attraction.
        end_attraction (str): The ending attraction.
        final_path_size (int): The length of each path in the population.

    Returns:
        initial_population (list): A list of paths (a permutation of attractions) of size = size.
    """
    initial_population = []
    attractions = list(graph.keys())
    
    # Remove start and end attractions from the list of attractions to visit
    attractions.remove(start_attraction)
    attractions.remove(end_attraction)

    for i in range(size):
        path = [start_attraction]  # Initialize path with start_attraction
        remaining_attractions = set(attractions)
        
        if random.random() < 1:
            # Random initialization
            path += random.sample(list(remaining_attractions), min(len(remaining_attractions), final_path_size - 2))
        else:
            # Nearest neighbor initialization
            while len(path) < final_path_size - 1 and remaining_attractions:
                last_attraction = path[-1]
                nearest_attraction = min(
                    remaining_attractions, 
                    key=lambda attraction: graph[last_attraction].get(attraction, float('inf'))
                )
                path.append(nearest_attraction)
                remaining_attractions.remove(nearest_attraction)
        
        path.append(end_attraction)  # Append the end attraction to the path
        initial_population.append(path)
    
    return initial_population



In [713]:
def calculate_distance_attraction(graph, attraction1, attraction2):
    """Calculates the distance between two attractions based on a graph

    Args:
        graph (dict): A dictionary representing the graph of attractions.
        attraction1 (str): The name of the first attraction.
        attraction2 (str): The name of the second attraction.

    Returns:
        distance (float): The distance between the two attractions. Returns float('inf') if there is no direct path.
    """
    # Return the distance from the graph if it exists, otherwise return infinity
    return graph.get(attraction1, {}).get(attraction2, float('inf'))

def CreateMatingPool(population, RankList, tournament_size=3):
    """
    Implements tournament selection for a genetic algorithm's parent selection.
    In each tournament, a subset of individuals is chosen at random, and the one with the best fitness
    (highest rank) is selected for the mating pool.

    Args:
        population (list): A list of paths (each path is a list of attractions) from which the mating pool is to be created.
        RankList (list): A list of tuples (index, fitness score) sorted in ascending order of fitness (lower is better).
        tournament_size (int): The number of individuals to be selected for each tournament. Default is 3.

    Returns:
        mating_pool (list): A list of paths selected from the population to form the mating pool.
    """
    
    matingPool = []
    
    for _ in range(len(population)):
        # Conduct a tournament
        tournament = random.sample(RankList, tournament_size)
        # Select the winner with the best fitness (lowest score)
        winner = min(tournament, key=lambda item: item[1])
        # Append the path corresponding to the winner to the mating pool
        matingPool.append(population[winner[0]])
                     
    return matingPool

In [714]:
def Crossover(Parent1, Parent2, Start_Index, End_Index):
    """Performs crossover between two parents and returns the child

    Args:
        Parent1 (list): The first parent (path of attractions)
        Parent2 (list): The second parent (path of attractions)
        Start_Index (int): The start index of the crossover section
        End_Index (int): The end index of the crossover section

    Returns:
        child (list): The child generated after crossover
    """
    child = []
    
    # Slice from Parent1 for the crossover section
    p1_section = Parent1[Start_Index:End_Index]
    
    # Remainder of the attractions that will be filled in from Parent2
    remainder = [attraction for attraction in Parent2 if attraction not in p1_section]
    
    # Construct the child: attractions before the crossover section from Parent2, 
    # the crossover section from Parent1, and the rest from Parent2
    child = remainder[:Start_Index] + p1_section + remainder[Start_Index:]
    
    # Ensure that all attractions are included exactly once
    # Find missing attractions and duplicate attractions in the child
    all_attractions = set(Parent1)
    child_attractions = set(child)
    missing_attractions = list(all_attractions - child_attractions)
    
    # Check for any missing attractions in the child and replace the duplicates with the missing attractions
    if missing_attractions:
        for i in range(len(child)):
            if child.count(child[i]) > 1:
                child[i] = missing_attractions.pop()
                
                if not missing_attractions:
                    break

    return child


In [715]:
def Mutate(Child, graph):
    """Mutates the child by performing one of two operations:
    1. Reversing the order of a random subset of the child, excluding the first and last elements.
    2. Replacing an attraction with an attraction not in the path, excluding the replacement of the first and last elements.

    Args:
        Child (list): The child to be mutated.
        graph (dict): A dictionary representing the graph of attractions.

    Returns:
        Child (list): The mutated child.
    """
    mutation_type = random.choice(['reverse', 'replace'])
    
    if mutation_type == 'reverse':
        # Reverse mutation
        if len(Child) > 3:  # Ensure there are elements to reverse (excluding the first and last)
            start_index = random.randint(1, len(Child) - 3)
            end_index = random.randint(start_index, len(Child) - 2)
            Child[start_index:end_index+1] = Child[start_index:end_index+1][::-1]
    
    elif mutation_type == 'replace':
        # Replacement mutation
        # Ensure there's an element to replace (excluding the first and last)
        if len(Child) > 2:
            # Get all possible attractions that are not in the child
            possible_replacements = set(graph.keys()) - set(Child)
            if possible_replacements:
                # Select a random attraction from the child to replace (excluding the start and end attractions)
                index_to_replace = random.randint(1, len(Child) - 2)
                # Select a random replacement attraction
                replacement_attraction = random.choice(list(possible_replacements))
                # Replace the attraction in the child
                Child[index_to_replace] = replacement_attraction
            
    return Child


In [716]:
def calculate_path_distance(graph, path):
    """Calculates the total distance of a path based on the graph distances.

    Args:
        graph (dict): A dictionary representing the graph of attractions.
        path (list): A list of attractions representing the path.

    Returns:
        total_distance (float): The total distance of the given path.
    """
    total_distance = 0
    for i in range(len(path) - 1):
        # Get the distance between the current attraction and the next, and add it to the total distance
        total_distance += graph[path[i]].get(path[i + 1], float('inf'))
    return total_distance


In [717]:
# Initial graph representing cities and distances between them
# Function to generate attraction names
def generate_attractions(num_attractions):
    return [f'Attraction{chr(65 + i)}' for i in range(num_attractions)]

# Function to generate a graph with random distances between attractions
def generate_graph(attractions):
    graph = {}
    for attraction in attractions:
        distances = {other_attraction: random.randint(10, 50) for other_attraction in attractions if other_attraction != attraction}
        graph[attraction] = distances
    return graph

# Generate 10 attractions
attractions = generate_attractions(10)
attractions_graph = generate_graph(attractions)
start_attraction = attractions[0]  # 'AttractionA'
end_attraction = attractions[-1]  # 'AttractionJ' (if 10 attractions)

print("Attractions:", attractions)
print("Start Attraction:", start_attraction)
print("End Attraction:", end_attraction)
print("Generated Graph:")
for attraction, distances in attractions_graph.items():
    print(f"{attraction}: {distances}")



Attractions: ['AttractionA', 'AttractionB', 'AttractionC', 'AttractionD', 'AttractionE', 'AttractionF', 'AttractionG', 'AttractionH', 'AttractionI', 'AttractionJ']
Start Attraction: AttractionA
End Attraction: AttractionJ
Generated Graph:
AttractionA: {'AttractionB': 10, 'AttractionC': 23, 'AttractionD': 11, 'AttractionE': 42, 'AttractionF': 39, 'AttractionG': 27, 'AttractionH': 32, 'AttractionI': 15, 'AttractionJ': 22}
AttractionB: {'AttractionA': 25, 'AttractionC': 10, 'AttractionD': 19, 'AttractionE': 48, 'AttractionF': 43, 'AttractionG': 50, 'AttractionH': 28, 'AttractionI': 19, 'AttractionJ': 38}
AttractionC: {'AttractionA': 44, 'AttractionB': 18, 'AttractionD': 19, 'AttractionE': 35, 'AttractionF': 12, 'AttractionG': 12, 'AttractionH': 30, 'AttractionI': 27, 'AttractionJ': 50}
AttractionD: {'AttractionA': 18, 'AttractionB': 23, 'AttractionC': 14, 'AttractionE': 34, 'AttractionF': 31, 'AttractionG': 19, 'AttractionH': 18, 'AttractionI': 34, 'AttractionJ': 13}
AttractionE: {'Attrac

In [744]:
initial_pop_size = 100
final_path_size = 8

initial_mutation_rate = 0.2
minimum_mutation_rate = 0.01

population = CreateInitialPopulation(initial_pop_size, attractions_graph, start_attraction, end_attraction, final_path_size)
len(population)

100

In [746]:
RankList = [(i, calculate_path_distance(attractions_graph, path)) for i, path in enumerate(population)]
RankList.sort(key=lambda item: item[1])
# First element of RankList is the best path
print("Best Path Distance (Initial):", RankList[0][1])

# Last element of RankList is the worst path
print("Worst Path Distance (Initial):", RankList[-1][1])

Best Path Distance (Initial): 123
Worst Path Distance (Initial): 259


In [747]:
mutation_rate = initial_mutation_rate

currentBestFitness = RankList[0][1] # The last element in the RankList is the best fitness
currentBestFitness

123

In [748]:
matingPool = CreateMatingPool(population, RankList)
len(matingPool)

100

In [750]:
newPopulation = []
for i in range(len(population)):
    # Select two parents from the mating pool
    Parent1 = matingPool[random.randint(0, len(matingPool)-1)]
    Parent2 = matingPool[random.randint(0, len(matingPool)-1)]
    
    # Perform crossover between the two parents
    crossover_start_index = random.randint(1, len(Parent1)-2)
    crossover_end_index = random.randint(crossover_start_index, len(Parent1)-2)
    Child = Crossover(Parent1, Parent2, crossover_start_index, crossover_end_index)
    
    # Mutate the child
    if random.random() < mutation_rate:
        Child = Mutate(Child, attractions_graph)
    
    # Add the child to the new population
    newPopulation.append(Child)
len(newPopulation)

100

In [752]:
RankList = [(i, calculate_path_distance(attractions_graph, path)) for i, path in enumerate(newPopulation)]
RankList.sort(key=lambda item: item[1])
currentBestFitness = RankList[0][1] # The last element in the RankList is the best fitness
# First element of RankList is the best path
print("Best Path Distance (Initial):", RankList[0][1])

# Last element of RankList is the worst path
print("Worst Path Distance (Initial):", RankList[-1][1])

Best Path Distance (Initial): 101
Worst Path Distance (Initial): 268


In [772]:
def run_genetic_algorithm(graph, initial_pop_size, start_attraction, end_attraction, final_path_size, generations, initial_mutation_rate, minimum_mutation_rate, debug = False):
    """Runs the genetic algorithm for itinerary planning.

    Args:
        graph (dict): The graph representing attractions and distances between them.
        initial_pop_size (int): The size of the initial population.
        start_attraction (str): The starting attraction.
        end_attraction (str): The ending attraction.
        final_path_size (int): The length of each path in the population.
        generations (int): The number of generations to run the algorithm.
        initial_mutation_rate (float): The initial mutation rate.
        minimum_mutation_rate (float): The minimum mutation rate.

    Returns:
        bestPath (list): The best path found by the genetic algorithm.
        bestFitness (float): The fitness of the best path.
    """
    bestPath = []
    bestFitness = float('inf')
    decay_rate = (initial_mutation_rate - minimum_mutation_rate) / generations
    population = CreateInitialPopulation(initial_pop_size, graph, start_attraction, end_attraction, final_path_size)

    for generation in range(generations):
        RankList = [(i, calculate_path_distance(graph, path)) for i, path in enumerate(population)]
        RankList.sort(key=lambda item: item[1])
        
        mutation_rate = initial_mutation_rate - (generation * decay_rate)
        mutation_rate = max(mutation_rate, minimum_mutation_rate)
        
        currentBestFitness = RankList[0][1]  # The first element in the RankList is the best fitness
        
        if currentBestFitness < bestFitness:
            if debug:
                print(f"At generation {generation}, the fitness improved from {bestFitness} to {currentBestFitness}")
            bestFitness = currentBestFitness
            bestPath = population[RankList[0][0]]
            
        matingPool = CreateMatingPool(population, RankList)
        
        newPopulation = []
        
        size = len(population)
        for i in range(size//2):
            # Select two parents from the mating pool
            Parent1 = matingPool[random.randint(0, len(matingPool)-1)]
            Parent2 = matingPool[random.randint(0, len(matingPool)-1)]
            
            # Select the crossover section ignoring the first and last attractions        
            start_index = random.randint(1, len(Parent1)-2)
            end_index = random.randint(start_index, len(Parent1)-2)
            
            child = Crossover(Parent1, Parent2, start_index, end_index)
            
            # Randomly mutate the child
            if random.random() < mutation_rate:
                child = Mutate(child, graph)
                
            newPopulation.append(child)
            
        # Retain size/2 best individuals from the previous population
        newPopulation += population[:size//2]
        population = newPopulation

    return bestPath, bestFitness

run_genetic_algorithm(attractions_graph, initial_pop_size, start_attraction, end_attraction, final_path_size, 100, initial_mutation_rate, minimum_mutation_rate, debug=True)


At generation 0, the fitness improved from inf to 118
At generation 2, the fitness improved from 118 to 104
At generation 8, the fitness improved from 104 to 101
At generation 17, the fitness improved from 101 to 93
At generation 20, the fitness improved from 93 to 86


(['AttractionA',
  'AttractionI',
  'AttractionB',
  'AttractionC',
  'AttractionG',
  'AttractionH',
  'AttractionF',
  'AttractionJ'],
 86)

In [ ]:
def create_graph_with_manhattan_distances(places):
    """
    Create a graph where nodes are attractions with provided latitudes and longitudes,
    and edges are the Manhattan distances between these attractions.

    Args:
        places (list of dicts): Each dict contains 'name', 'lat', and 'long' keys for an attraction.

    Returns:
        dict: Graph representing attractions and Manhattan distances between them.
    """
    attractions = [place['name'] for place in places]  # Extracting attraction names
    graph = {}

    # Generate distances between each pair of attractions
    for i, place1 in enumerate(places):
        attraction1 = place1['name']
        graph[attraction1] = {}
        for j, place2 in enumerate(places):
            attraction2 = place2['name']
            if i != j:
                # Calculate Manhattan distance
                distance = abs(place1['lat'] - place2['lat']) + abs(place1['long'] - place2['long'])
                graph[attraction1][attraction2] = distance

    return graph

# Example usage:
places = [
    {'name': 'AttractionA', 'lat': 40.7128, 'long': -74.0060},
    {'name': 'AttractionB', 'lat': 34.0522, 'long': -118.2437},
    # ... add more places
]
attractions_graph = create_graph_with_manhattan_distances(places)

for attraction, distances in attractions_graph.items():
    print(f"{attraction}: {distances}")
